# Multi-class tasks: the case of topic classification

In one of the previous modules you learned about binary classification and how to implement it using machine learning.
However, some classification tasks involve more than just two classes.
In this module, you will learn how to address such tasks specifically.


## Topic classification

Consider a program that collects articles from different websites.
The articles span four broad subjects: _politics_, _science_, _tech_, and _sports cars_.
When you open the news feed, all the articles are mixed together.
What if you want to sort them by topic?

Some RSS feed managers, such as Google News, implement this idea.

![Google News with topic classification](figures/google_news_topics.png)


## 20 Newsgroups dataset

In this module, you will use the [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/) dataset, a collection of approximately 20,000 newsgroup documents.
It is widely used by the NLP community to develop and test topic classification models, as well as text classification and text clustering algorithms.

The data comes from 20 different newsgroups with different topics – which can be grouped into broad topic categories.
In order to make the task more approachable, – you are still learning after all, – you will focus on four topics from four newsgroups:

- **politics** from `talk.politics.misc`,
- **electronics** from `sci.electronics`,
- **Macintosh hardware** from `comp.sys.mac.hardware`, and
- **automobiles** from `rec.autos`.

Afterwards, when you feel comfortable with the implementation of the classifier, you can experiment with a different selection of topics.


## Classifier

Topic classification is more complicated than sentiment analysis: many words can appear in multiple topics – e.g., “hardware” can appear in articles about either _Macintosh hardware_ or _electronics_.
Thus, the frequency of words and the probability of their occurrence in texts on different topics are a the most predictive type of feature.
You will use it in your classifier implementation along with the [multinomial naïve Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) model.

Internally, the algorithm you will implement represents each topic (class) $y$ as a vector $\theta$ of size $n$, where $n$ is the number of words taken into account for classification, the _vocabulary size_.
The vectors are composed of $n$ estimates $\theta_{y_i}$ of the probabilities $P(x_i|y)$ that the feature $i$ appears in texts on the topic $y$.
These estimates can be computed from the data by counting the occurrences ($N_{y_i}$) of a specific feature ($i$) in training texts on a specific topic ($y$).

\begin{equation}
\theta_{y_i} = \frac{ N_{y_i} }{ N_y }
\end{equation}

The number $N_{y_i}$ is calculated as the sum across texts on the topic $y$ of the uses of the word $i$.
The number $N_y$ is the total count of all features for the class $y$: the sum $\sum_{i=1}^{|T|} N_{y_i}$, where $T$ is the total number of topics.

### Example

Consider a simple vocabulary of just five words: “computer”, “motor”, “show”, “system”, and “visit”.
In this simple example, all articles only consist of these five words – or, to be more realistic, you only care about these five words.

Topic vectors (the $\theta$s) have five elements (dimensions), one for each of the words in the vocabulary.
You can expect the word “system” to appear in texts on _politics_ (e.g., “political system”) as well as on _Macintosh hardware_ (e.g., “operating system”).
However, their frequency and the frequency of other words will vary.
You can expect vectors such as:


 $\theta_{\mathit{topic}_\mathit{feature}}$|    computer   |   motor   |   show   |   system   |   visit
-------------------------------------------|:-------------:|:---------:|:--------:|:----------:|:---------
**politics**                               |       0.0     |     0.0   |   0.1    |   0.4      |   0.5  
**electronics**                            |       0.4     |     0.0   |   0.2    |   0.4      |   0.0
**Macintosh hardware**                     |       0.5     |     0.0   |   0.1    |   0.4      |   0.0
**automobiles**                            |       0.1     |     0.5   |   0.2    |   0.1      |   0.1  


### Smoothing

The table above contains null probabilities for the chance of some words occurring in documents on some topics (e.g., "computer" in _politics_).
This can happen if, during the algorithm's training, the word “computer” never appears in articles about politics.

However, human language is very diverse. And it is impossible to collect reliable statistics on the use of **all** words in **all** topics.
As a result, in practice, you should avoid assigning null probabilities.
This is why the formula is generally adjusted with a term $\alpha$:

\begin{equation}
\theta_{y_i} = \frac{ N_{y_i} + \alpha}{N_y + \alpha \times n}
\end{equation}

Using $\alpha = 1$ means that you increment all word counts by one; the count of unseen words becomes 1.
In the formula above, $n$ is the total number of features, or the size of the vocabulary.
Since you increment the counts for each of the $n$ features by $\alpha$ in numerator, adding $\alpha \times n$ to denominator makes sure you still get
proper probability distribution (i.e., $\theta_{y_i}$ is still within the range of $[0, 1]$).
This adjustment is called _Laplace smoothing_ or _add-one smoothing_.

After smoothing, the table above becomes:

 $\theta_{\mathit{topic}_\mathit{feature}}$|    computer   |   motor   |   show   |   system   |   visit
-------------------------------------------|:-------------:|:---------:|:--------:|:----------:|:---------
**politics**                               |      0.05     |    0.05   |  0.07    |   0.37     |   0.46  
**electronics**                            |      0.37     |    0.05   |  0.16    |   0.37     |   0.05
**Macintosh hardware**                     |      0.46     |    0.05   |  0.07    |   0.37     |   0.05
**automobiles**                            |      0.07     |    0.46   |  0.16    |   0.07     |   0.07  

Note that the order of the word frequencies is the same before and after smoothing.


## Implementation with `sklearn`

You will use [`sklearn`](http://scikit-learn.org/stable/index.html) for the implementation of the multinomial naïve Bayes classifier.

* The `sklearn` library is a useful toolkit for data mining and data analysis. It is useful to add it to your belt.
* It includes a range of classification techniques and data processing tools.
* It has exactly the right features for analysing the 20 Newsgroups dataset.

### Importing the libraries

Start by importing the necessary libraries with the following code:


In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

Several remarks are in order.
First, note that `sklearn` includes direct support for the 20 Newsgroups dataset.
Second, remember that the `TfidfVectorizer` import is for _Term Frequency times Inverse Document Frequency_ functionality.
Third, the module `numpy` is imported `as` `np` which means that the identifier `np` in the rest of the code refers to `numpy`. This is just for convenience: to make code shorter.

### Step 1: Loading the data

The next step is to access and load the dataset.
The `fetch_20newsgroups` function imported earlier helps you access the full dataset.

Write a function `load_dataset` that wraps calls to `fetch_20newsgroups` and passes specific arguments:


In [2]:
def load_dataset(sset, cats):
    if cats==[]:
        newsgroups_dset = fetch_20newsgroups(subset=sset,
                          remove=('headers', 'footers', 'quotes'),
                          shuffle=True)
    else:
        newsgroups_dset = fetch_20newsgroups(subset=sset, categories=cats,
                          remove=('headers', 'footers', 'quotes'),
                          shuffle=True)
    return newsgroups_dset

The argument `remove` simplifies the data by removing specific parts (here, the headers, footers and quote blocks).

Use the function in your program: set categories and pass them to the `load_dataset` function:


In [3]:
categories = ['talk.politics.misc', 'sci.electronics', 'comp.sys.mac.hardware', 'rec.autos']
newsgroups_train = load_dataset('train', categories)
newsgroups_test = load_dataset('test', categories)

In [4]:
type(newsgroups_train)


sklearn.utils.Bunch

### Step 2: Inspecting the data

Once again, you should check that the data loaded correctly and you should get familiar with its format.
Create a function `inspect_data` which prints out informative properties of the dataset

* the target names
* the shape of filenames
* the shape of target
* what target looks like
* ...


In [5]:
type(newsgroups_train.target) 

numpy.ndarray

In [6]:
# add your code here

###https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

def inspect_data(dataset):
    print(list(dataset.target_names))
    print(dataset.filenames.shape)
    print(dataset.target.shape)
    print(dataset.target[:10])
    
newsgroups_test.target
type(newsgroups_test.data)

list

The different elements that are printed are the different constituents of the dataset, as loaded by `sklearn`. You can check online the documentation for [the dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) or you can use the Python interpreter to explore it interactively.

Call the function and read the output. Note that labels are represented as numerical values (`0` for `talk.politics.misc`, `1` for `sci.electronics`, etc.).


In [8]:
# add your code here
#categories = ['talk.politics.misc', 'sci.electronics', 'comp.sys.mac.hardware', 'rec.autos']

######## BETTER --- to sort the categories as it will be stored internally in an alphabetical order anyway!

categories = sorted(['talk.politics.misc', 'sci.electronics', 'comp.sys.mac.hardware', 'rec.autos'])


newsgroups_train = load_dataset('train', categories)
inspect_data(newsgroups_train)
newsgroups_test = load_dataset('test', categories)
inspect_data(newsgroups_test)


['comp.sys.mac.hardware', 'rec.autos', 'sci.electronics', 'talk.politics.misc']
(2228,)
(2228,)
[0 3 0 1 3 3 1 3 2 1]
['comp.sys.mac.hardware', 'rec.autos', 'sci.electronics', 'talk.politics.misc']
(1484,)
(1484,)
[3 1 0 1 1 0 0 2 0 2]


### Step 3: Feature extraction

With the data loaded, you can now extract features from the texts.
As pointed above, for multi-class classification, the features are words frequencies and each topic is represented as a vector of the word frequencies. Let's look once again into *tf-idf* technique.

#### Frequency vs count

Remember that for text analysis simply counting the occurrences of words is not good enough.
An immediate issue is that the longer a text, the higher the word counts in general. Conversely, the shorter the text, the lower the word counts.

To solve this issue, you need to divide the word counts by the total number of words in the text.
This technique is known as *Term Frequency* or *tf*.

#### Common words

Another issue arising in text analysis is that even some meaningful words appear in too many texts regardless of a topic – e.g., “issue”, “document”, “human”.

For classification, it is important to find distinctive features: words that appear very frequently in some topics and very rarely in others.
To downweigh the words that occur in every topic, you need to divide the term frequency in the topic by the frequency across all topics.
The technique is called *Term Frequency times Inverse Document Frequency*, or *_tf–idf_*.

In `sklearn` you can use it directly.

Define a function `text2vec` taking a training set and a test set.

* apply a `TfidfVectorizer` with english stop words, fit it on the training set and apply it on both
* return the vectorizer as well as the transformed data. 


In [10]:
# add your code here
def text2vec(train_set, test_set):
    vectorizer = TfidfVectorizer(stop_words = 'english') # Convert a collection of raw documents to a matrix of TF-IDF features.
    vectors_train = vectorizer.fit_transform(train_set.data)
    vectors_test = vectorizer.transform(test_set.data) ################## no FIT in the test set, only in the train set!!!
    return (vectorizer, vectors_train, vectors_test)   ################## as we will eveluate our model in TEST based on what
                                                       ################## we trained on the TRAIN SET


First a vectoriser is initialised. Then, it is used to extract features from the dataset.

Use this function and print information about the results.
In addition, let's estimate the number of non-zero components in the vectors – that is, the words from the joint training and test set vocabulary that occur in the texts of both subsets.


In [11]:
categories = ['talk.politics.misc', 'sci.electronics', 'comp.sys.mac.hardware', 'rec.autos']
newsgroups_train = load_dataset('train', categories)
newsgroups_test = load_dataset('test', categories)

vectorizer, vectors_train, vectors_test = text2vec(newsgroups_train, newsgroups_test)

print("\nData format:")
print(vectors_train.shape)
print("Non-zero components estimate:")
print(vectors_train.nnz / float(vectors_train.shape[0]))

print("\nData format:")
print(vectors_test.shape)
print("Non-zero components estimate:")
print(vectors_test.nnz / float(vectors_test.shape[0])) ##### if it is 20 or 30%, the test set is not good as it does not
                                                       ##### represent the train set

print('------') ################################################# included
print(vectors_train.nnz)
print(vectors_train.shape[0])
print(vectors_train.shape)
print(vectors_train)



Data format:
(2228, 21639)
Non-zero components estimate:
52.012567324955114

Data format:
(1484, 21639)
Non-zero components estimate:
44.296495956873315
------
115884
2228
(2228, 21639)
  (0, 9965)	0.08520864049746116
  (0, 10175)	0.06761187080335423
  (0, 2868)	0.08309262622595794
  (0, 18593)	0.06214733264462163
  (0, 8244)	0.10506063386055675
  (0, 12473)	0.0857849428378622
  (0, 7018)	0.14788831826196372
  (0, 9777)	0.0650365035623811
  (0, 20771)	0.07138269848838813
  (0, 3212)	0.07995776716450928
  (0, 17312)	0.05635066047989126
  (0, 6848)	0.12702864149515555
  (0, 19655)	0.06601701634479967
  (0, 21421)	0.07878341015656297
  (0, 15594)	0.2540572829903111
  (0, 5526)	0.11604463767785615
  (0, 19333)	0.09736700132270687
  (0, 15094)	0.09509934409651985
  (0, 3318)	0.08260500104945903
  (0, 16099)	0.10319417699605977
  (0, 13313)	0.12702864149515555
  (0, 16480)	0.09619264431476056
  (0, 19419)	0.07804622757305782
  (0, 12147)	0.05925816172686216
  (0, 3939)	0.12060341115425738
 

The pairs in the output indicate the number of texts in each subset and the vocabulary size.
Note that test vectors contain more zero elements – they are sparser.


### Step 4: Multi-class classification

With the feature vectors, you can now implement a classification algorithm.
Add the function `classify` to your program:

* arguments: `vectors_train`, `vectors_test`, `train_set` and `clf` (an untrained classifier)
* fit the classifier on the training data and `train_set.target`
* predict and return the prediction


In [13]:
# add your code here
def classify(vectors_train, vectors_test, train_set, clf):
    clf.fit(vectors_train, train_set.target)
    predictions = clf.predict(vectors_test)
    return predictions


This function takes four arguments:

- `vectors_train` and `vectors_test` as returned by the vectoriser,
- `train_set` for the training set, and
- `clf` for the classifier.

The classifier is trained against (or fitted to) the training data.
Then, the trained classifier is used on the test data.


### Step 5: Classification evaluation

So far so good: you should be able to run the code above and method `classify` will assign the texts from your test set to one of the four topics each.
However, how can you find out if its predictions are any good?
Let's call on the `sklearn`'s `metrics.classification_report` to check how well the classifier performs:


In [14]:
# add your code here
predictions = classify(vectors_train, vectors_test, 
                       newsgroups_train, clf=MultinomialNB(alpha=.01))
full_report = classification_report(newsgroups_test.target, 
                                    predictions, target_names=newsgroups_test.target_names)
print(full_report)


                       precision    recall  f1-score   support

comp.sys.mac.hardware       0.86      0.83      0.84       385
            rec.autos       0.77      0.87      0.82       396
      sci.electronics       0.84      0.76      0.80       393
   talk.politics.misc       0.86      0.88      0.87       310

             accuracy                           0.83      1484
            macro avg       0.84      0.83      0.83      1484
         weighted avg       0.83      0.83      0.83      1484



Note that `sklearn` does most of the heavy lifting: all you need to do is call its function with arguments relevant to your problem and pass results around to the next function.
This is a blessing because you do not need to worry about the details of the implementation, but also a curse because you are entirely dependent on the library.
Thus, it is important to read the [library's documentation](http://scikit-learn.org/stable/modules/classes.html).


#### Understanding the evaluation report: accuracy

To make more sense of the results reported above, consider first a toy example of classification with two topics: _politics_ (with 90 texts), and _sports_ (with 10).
Suppose the classifier report is a table as below:



Topics                   | **politics (actual)**|  **sports (actual)**| **Total**
--------------------     |:--------------------:|:-------------------:|:---------:
**politics (predicted)** |          85          |           6         |   91
**sports (predicted)**   |          5           |           4         |   9
**Total**                |          90          |           10        |   100

The classifier predicts that there are 91 texts on _politics_ and 9 on _sports_.
This gives an accuracy of 89%.

However, this accuracy is not a good measure of how well – or in this case badly – the classifier performs.
Indeed, consider that 90% of texts are about _politics_: a better classifier than the imaginary one above would simply assign all texts to _politics_ and get an accuracy of 90% by doing nothing at all!

To better understand the problems of the imaginary classifier above, you must consider additional details: how it performs on each class of text.

#### Understanding the evaluation report: true/false positives/negatives

The classifier's predictions for any class can be represented as a _confusion matrix_.
For example, the confusion matrix for the class $c_1$ is as follows:

Classes              |  $c_1$ (actual)  |   $c_2$ (actual)
------------------   |:----------------:|:-----------------:
$c_1$ (**predicted**)|  true positive   |  false positive
$c_2$ (**predicted**)|  false negative  |  true negative

* The **true positives** ($\mathit{tp}$) are the texts that actually belong to $c_1$ and are classified as $c_1$.
* The **false positives** ($\mathit{fp}$) are the texts that actually belong to $c_2$ but are classified as $c_1$.
* The **false negatives** ($\mathit{fn}$) are the texts that actually belong to $c_1$ but are classified as $c_2$.
* The **true negatives** ($\mathit{tn}$) are the texts that actually belong to $c_2$ and are classified as $c_2$.

These four numbers exhaustively describe the classifier's performance on texts of $c_1$.

#### Understanding the evaluation report: metrics

The four numbers $\mathit{tp}$, $\mathit{fp}$, $\mathit{fn}$, and $\mathit{tn}$ are used to compute more manageable evaluations.
Specifically, four distinct metrics are computed from them.

* **Precision**: $P = \frac{\mathit{tp}}{\mathit{tp} + \mathit{fp}}$

    The precision is the proportion of true positives among the texts that the classifier predicts belong to $c_1$.
    It is a measure of the classifier's _reliability_: how trustworthy the predictions are.

* **Recall**: $R = \frac{\mathit{tp}}{\mathit{tp} + \mathit{fn}}$

    The recall is the proportion of true positives among all the texts that actually belong to $c_1$.
    It is a measure of your classifier's _coverage_: its ability to find all the relevant cases in the data.

* **F1 score**: $\mathit{F_1} = 2 \times \frac{\mathit{precision} \times \mathit{recall}}{\mathit{precision} + \mathit{recall}}$

    The $F_1$ score is the harmonic mean of the precision and recall metrics.
    You can treat it as a measure that reflects your classifier's ability to find all the relevant cases and only those.

* **Accuracy**: $\mathit{Acc} = \frac{\mathit{tp} + \mathit{tn}}{\mathit{tp} + \mathit{tn} + \mathit{fp} + \mathit{fn}}$

    The accuracy is the proportion of correct results.


These metrics indicate the different shortcomings of a classifier. E.g., for the imaginary classifier above, the precision, recall and $F_1$ score are:

Topics | _politics_  | _sports_  | Average (macro)
------ |:-----------:|:---------:|:----------------:
P      |  0.930      |   0.440   |   0.685
R      |  0.940      |   0.400   |   0.670
$F_1$  |  0.935      |   0.420   |   0.678

The classifier scores low on all metrics for the sports category.
In fact, the precision is below 0.5: the classifier makes more erroneous guesses than correct guesses when presented with sports-related text.


#### Reading the evaluation report

Running the classifier code with report printing gives the following output:

~~~~
Full report:
                       precision    recall  f1-score   support

comp.sys.mac.hardware       0.86      0.83      0.84       385
            rec.autos       0.77      0.87      0.82       396
      sci.electronics       0.84      0.76      0.80       393
   talk.politics.misc       0.86      0.88      0.87       310

          avg / total       0.83      0.83      0.83      1484
~~~~

The `support` column simply reports the number of texts on each topic.
As you can see the distribution of texts across four topics is quite balanced, and so is the classifier's performance.


**Bonus exercise:**

>Use [`sklearn`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)'s functionalities to print the confusion matrix of your classifier for some of the available classes.


#### Most predictive features

You can access the most predicitve feature of a vectoriser.
The function below, `show_top10`, extracts this information. 
You can also add the call to this function to your `classify` method:


In [22]:
def show_top10(classifier, categories, vectorizer):
    feature_names = np.asarray(vectorizer.get_feature_names())
####    for i, category in enumerate(categories):
    for i, category in enumerate(sorted(categories)): ##################### to FIX the order of the CATEGORIES!!!
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: %s" % (category, " ".join(feature_names[top10])))

def classify(categories, vectors_train, vectors_test, train_set, clf):
    clf.fit(vectors_train, train_set.target)
    predictions = clf.predict(vectors_test)
    show_top10(clf, categories, vectorizer)
    return predictions

Now, you can run the classifier and get the most predictive words per topic along with the full report:


In [23]:
# add your code here
predictions = classify(categories, vectors_train, vectors_test, 
                       newsgroups_train, clf=MultinomialNB(alpha=.01))
full_report = classification_report(newsgroups_test.target, 
                                            predictions, target_names=newsgroups_test.target_names)
print(full_report)


comp.sys.mac.hardware: card use monitor know does problem thanks drive apple mac
rec.autos: dealer know don new good engine just like cars car
sci.electronics: thanks circuit good don used power know does like use
talk.politics.misc: know men clinton tax president just think don government people
                       precision    recall  f1-score   support

comp.sys.mac.hardware       0.86      0.83      0.84       385
            rec.autos       0.77      0.87      0.82       396
      sci.electronics       0.84      0.76      0.80       393
   talk.politics.misc       0.86      0.88      0.87       310

             accuracy                           0.83      1484
            macro avg       0.84      0.83      0.83      1484
         weighted avg       0.83      0.83      0.83      1484



The function stores features in a `numpy` array. It then iterates over the categories, sorting the features by their predictive power each time.


# Language modelling and language generation

In all previous modules you worked on language analysis and tried to teach computer how to "understand" language.
In this module you will look into an opposite task: how to teach computer create or generate language.

In essence, as before with the machine learning approaches, computer will learn from experience:
having seen many good examples of language use it will try to replicate the most probable phrases.


## Text prediction

Language generation is widely used in text prediction applications, for example on your mobile phones.

![Text prediction](figures/text_prediction.png)


**Quiz:**

>Can you predict the next character(s) that should follow given string?

>* _boo_ ?
>* _bootc_ ? ? ?

>Can you predict the next word?

>* _Happy_ ? ? ? ? ? ? ? ?
>* _Good_ ? ? ? ?
>* _Have a good_ ? ? ? ? ?

**Answer:**
>Although you might have come up with different suggestions, with high probability the words that came to your mind are:

>* _boo**t**_
>* _bootc**amp**_
>* _**birthday**_
>* _**luck**_
>* _**night**_

The examples above show that when presented with such riddles we implicitly rely on our observations and knowledge about language.
For example, note that:

* It is harder to predict the next character or word based on shorter string: for instance, _bo_ can be followed by many more other characters than _boo_.
* Some character or word combinations make the prediction almost certain: for instance, once you see _bootc_ or even _bootca_, you are left with a single option of _bootcamp_.
* Even when the previous characters or words (called _context_) do not reliably predict the next character or word, they help to eliminate the improbable options:
for instance, _a_ is very unlikely to follow _boo_, and _computer_ is quite unlikely to follow _Happy_.
* The examples above use unidirectional context and rely on what comes before the character or word.
Bidirectional context might help further: it is easier to guess _bootcamp_ if you see _boo_ ?? _amp_ then if you see only _boo_.
It is also easier to predict _luck_ if you see "_Good_ ? ? ? ? *!*".

All these observations, expressed in precise statistical terms, help applications on your computers and mobile phones predict following characters and words.
In this module you will use Language Models and implement one in Python to build your own text prediction and text generation application.


## Language Models

If you want to convert the observations above into more precise statistical form, you will see that one can use _conditional probabilities_ estimated from actual texts to predict the next characters and words.

For example, you have a context of "_Have a good_" and you would like to predict which word should follow:

![Language Model](figures/LM.png)

You can look into a big collection of English texts and estimate how probable each of the words is given the context.
In other words, you estimate the conditional probabilities $P(day|\text{Have a good})$, $P(night|\text{Have a good})$, ..., $P(bootcamp|\text{Have a good})$.
Recall that you have already come across conditional probabilities when you used Naive Bayes classifier.
These probabilities can be directly estimated from the data as 

\begin{equation}
P(day|\text{Have a good}) = \frac{num(\text{Have a good day})}{\sum_{\mathit{X} \in \mathit{vocabulary}} num(\text{Have a good X})}
\end{equation}

and so on.
In other words, the probability of the word "_day_" following the context "_Have a good_" equals the number of times you see the phrase "_Have a good day_"
divided by the number of times you see a phrase "_Have a good_" followed by all the different words X that can be used in this context.

Once the conditional probabilities are estimated, you can pick the most probable next word directly comparing the probability estimates for the word candidates.
For example, if you get the following probabilities:

![Language Model with probabilities](figures/LM_prob.png)

you will choose "_day_" as the next word, since $P(day|\text{Have a good}) > P(night|\text{Have a good})$ > ... > $P(bootcamp|\text{Have a good})$.

This approach is called *Language Model* and it can be extended to estimate the probabilities of whole sentences.
For example, to estimate how probable sentence "_Have a good day_" is you use _chain rule_ and multiply the probabilities of the word sequences in the sentence:
$P(\text{Have a good day}) = P(Have) \times P(a|Have) \times P(good|\text{Have a}) \times P(day|\text{a good})$.

You might have noticed that the formula above uses different number of words in the conditional part:
it is one word for $P(a|Have)$ and two for the words that come later.
The reason for that is that when we start a sentence, we don't yet have enough context to condition upon.
Once we reach the word "_a_" we can use one previous word "_Have_", and when we move further in the word sequence we can start using longer previous context as condition.

Why then don't we use $P(day|\text{have a good})$ and rather stick to two previous words only?
The reason here is that the longer the phrase gets, the less frequent it is in the data: you see "_a good_" more often than "_Have a good_" simply because the former is a subset of the latter.
After certain number of words in the context the probabilities become unreliable, and it is usual to stick to a limited number of words from the previous context to estimate the probabilities.

The number of words or symbols used by the Language Model define its order:
the model which uses one previous symbol (character or word) is called _unigram_ model,
the one that uses two previous symbols – _bigram_, three symbols – _trigram_, and so on.
In practice, trigram models are used most frequently.

In this module, you will implement your own character-based language model and apply it to generate new text.


## Building a Language Model in Python

Let's start with the import statements:


In [19]:
from collections import defaultdict, Counter #1
from numpy import cumsum, sum, searchsorted #2
from numpy.random import rand #3

1. Import Python [`defaultdict`](https://docs.python.org/2/library/collections.html) and [`Counter`](https://docs.python.org/2/library/collections.html).
You will use them to estimate the probabilities.
2. Import numpy's [`cumsum`](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.cumsum.html),
[`sum`](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html) and
[`searchsorted`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.searchsorted.html).
You will use them to retrieve the predictions.
3. Import [`rand`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html)
which you will use to generate random numbers.

Next, let's define the functionality of the language model.
We will define Language Model as a stand-alone Python [class](https://docs.python.org/3/tutorial/classes.html): `class LanguageModel(object)`


This will allow us to easily call the methods of the `LanguageModel` from other python scripts.
To use `LanguageModel` from other scripts you'll need to create `LanguageModel` _objects_ by typing in `objname = LanguageModel(argument)`.
_Argument_ is some property of the object that will define your language model.
Let's use the order of the model as such important property of the language model object.


In [15]:
class LanguageModel(object):
    def __init__(self, order=1): #1
        '''Initializes a language model of the given order.'''
        self._transitions = defaultdict(int) #2
        self._order = order

1. The `init` method instantiates an object of the class using the provided arguments.
In this case, there's only one argument that you specify – the order of the model, or how far back in the context it will look.
By default, you say that the model will rely on _one_ previous symbol (character).
`self` here refers to the instance of the class `LanguageModel` object.
2. As one of the _fields_ a language model object will keep the transitions from the context to the next character.
These transitions will be stored in the Python's `defaultdict`.

The objects of class `LanguageModel` should be able to:

* _estimate_ the transition probabilities from the context of given order to the next characters based on some training text;
* _predict_ the next character based on any new context;
* _generate_ a whole sequence using its predictions.

To this end let's add three public methods to the class `LanguageModel` that will provide all the functionality described above:

* `train` to estimate the probabilities;
* `predict` to predict next character;
* `generate` to generate a whole sequence of characters.


In [16]:
class LanguageModel(object):
    def __init__(self, order=1): #1
        '''Initializes a language model of the given order.'''
        self._transitions = defaultdict(int) #2
        self._order = order
        
    def train(self, sequence): #1
        '''Trains the model using sequence.'''
        self._symbols = list(set(sequence))
        for i in range(len(sequence)-self._order):
            self._transitions[sequence[i:i+self._order], sequence[i+self._order]] += 1 #2

    def predict(self, symbol): #3
        '''Takes as input a string and predicts the next character.'''
        if len(symbol) != self._order:
            raise ValueError('Expected string of %d chars, got %d' % (self._order, len(symbol))) #4
        probs = [self._transitions[(symbol, s)] for s in self._symbols]
        return self._symbols[self._weighted_pick(probs)] #5

    def generate(self, start, n): #6
        '''Generates n characters from start.'''
        result = start
        for i in range(n):
            new = self.predict(start) #7
            result += new
            start = start[1:] + new #8
        return result

    @staticmethod
    def _weighted_pick(weights): #9
        '''Weighted random selection returns n_picks random indexes.
        The chance to pick the index i is given by weights[i].'''
        return searchsorted(cumsum(weights), rand()*sum(weights))

1. The method `train` learns the transition probabilities from a text, which is represented as a string and provided to the method as an argument `sequence`.
2. The `transitions` dictionary keeps the number of times the context of the specified order (length) is followed by the current character.
3. The method `predict` chooses the most probable character given the preceding one(s).
The preceding one(s) are provided to the method as an argument `symbol`.
4. If the length of the provided sequence of the previous characters doesn't match the language model order, report an error.
5. Return the character with a given probability.
6. The method `generate` allows you to generate a sequence of a specified number (`n`) of characters.
7. For that, it calls on the `predict` method providing it with the context `start`.
8. It moves the context character by character specified number of `n` times, thus allowing you to generate `n` new characters.
9. Method `weighted_pick` provides search functionality for the probabilities.


## Using your Language Model in practice

Now let’s test how the language model works in practice.
You'll start by trying to generate texts using famous books to train the language model.
In that case, you should expect to get the generated text that is quite similar in style to the text of the book you trained your language model on.

Let's import [urllib](https://docs.python.org/3.0/library/urllib.request.html) that will allow you to access texts from online resources. You can download a book from a collection of [Gutenberg Project books](https://www.gutenberg.org) available online. Let's start with _Robinson Crusoe_:


In [17]:
import requests

# add your code to load the page https://www.gutenberg.org/files/521/521-0.txt
in_text = requests.get('https://www.gutenberg.org/files/521/521-0.txt').text
        
print(type(in_text))
print(len(in_text))
print(in_text[:75])


<class 'str'>
655616
ï»¿The Project Gutenberg eBook, The Life and Adventures of Robinson Crusoe,


This piece of code downloads Robinson Crusoe and puts all the contents of this book in a (very long) string.
Let's now train your language model and generate new text:


In [22]:
model = LanguageModel(order=1) #1
model.train(in_text)
print (model.generate('t', 100)) #2

th tthe lllthad hed th whakn he anenâ and te ad d lean th aney t fomecatour,
s, wrofissuts wa dimy


1. Here, you use a very simple language model that relies on **one previous character** to generate the **next character**.
2. You ask the model to generate a sequence of 100 characters with the starting character "_t_".

If you run this code, you will get something like that:

~~~~
t s louma dorg s d axchoy theabexand Brabofoy s they anore s p, gh kind thate, me che weraumen 2.
~~~~

This doesn’t look readable. Can you see the possible reasons for that?

Let's try increasing the order to 2:


In [23]:
model = LanguageModel(order=2) #1
model.train(in_text)
print (model.generate('if', 100)) #2

if the frovely
up of mign th, earst my could
but in the beggertiony carry a Procknound to, had hor? 


1. Order increased to **2 previous characters**.
2. Note that the starting context now should also contain at least two characters.

Now, if you run this code you might get the following output:

~~~~
if all thende, I lion, any ths’s
as; an une
ship, whistrould I seep ints of inved; ot threence be
~~~~


This is still not quite readable, but you can notice a clear improvement: the words start resembling proper English.
Let’s increase the order to 4:


In [24]:
model = LanguageModel(order=7) #1
model.train(in_text)
print (model.generate('imagine', 100)) #2

imagined: and bury the dispute, for theirs.  In a word, I told him to know what indefatigable part fire, th


This time the output looks much better:

~~~~
your I would gest out of trees, or board of thes,
and I might,
as for, he wretchest, thought me from s
~~~~

Nearly all the words generated by the model can be recognised as English words.
You can now push your model further and increase the order to 6.
Alternatively, you can also change the training data and use some Shakesperean text, for example "_Romeo and Juliet_":


In [25]:
book = 'http://www.gutenberg.org/cache/epub/1112/pg1112.txt' # Romeo and Juliet
in_text = requests.get(book).text
        
model = LanguageModel(order=6)
model.train(in_text)
print (model.generate('young ', 1000))

young Romeo is banishment.

  Prince will to go.

  Lady. Here we have took.

  Friar. O woeful sympathy!
    A choking gave, that a head at the phrase "Project Gutenberg Literary Archive Foundation, modified in any man secret and find out logs
    When I hope thought him. An eagle, madam!
    Will none, she is come above at night, you in writing with all,
    And it missheath; but surcease;
    Ere one can any money, on my children of mine of us, look on here dead, deceas'd; she's debt.                Exit.

  Nurse. Come, death, shall smooth thy will,
    was this discovered and the tomb;
    I come. What shall be spent.
    A sin-absolv'd.

    Where honour man shall have in thine. Things true.

  Ben. Two, two! a shirt and darkness lengthens Romeo.

  Cap. Young men's rattling both!
    Come you?

  Jul. How now, my liege, my wit! I will still the Project Gutenberg-tm
electronic works in any work of heaven,
    Romeo and Men.

  Ben. By my coz.

  Cap. All 


You might expect to get some interesting output now.

As you can see, the model is now able to generate text that resembles the early modern English used by Shakespeare.
It has even been able to generate the structure of a dialogue between the characters.

The model is still far from generating sensible piece of text – you may notice that it doesn't always respect all the conventions of English grammar and it doesn’t yet write with a purpose. It might also be not as good and convincing as some other [examples of more advanced LMs generating natural text](https://twitter.com/deepdrumpf).
Nevertheless, the simplicity of the model with respect to the complexity of its behaviour provides you with a non-trivial fresh perspective on the way language can be generated computationally.


## Language Model for review generation

Now let's use the `LanguageModel` from this module and `imdb_dataset/pos/` texts as your training data
to automatically generate positive movie reviews. You can also experiment with negative movie reviews.


In [26]:
import os

def read_data(a_dir):
    train_data = ""
    file_list = os.listdir(a_dir)
    for a_file in file_list:
        f = open(a_dir + a_file, 'r')
        train_data += f.read() + " "
        f.close()
    return train_data.strip()

# You need to read in all reviews as one string of text
# to train the language model on
train_data = read_data("data/imdb_dataset/pos/")

model = LanguageModel(order=8)
model.train(train_data)
print (model.generate('the best', 280)) # let's generate a long tweet up to 280 characters

the best version of "A Christmas special relationship between The Empire's all-new half-completely strangle her while Lando Calrissian and women, black and white of heart, it was totally redundant and hope that turns on Luke. There are a good dramas, the audience-pleaser after" Hollywood


## Evaluation of Language Models

For all of the previous models we used some evaluation techniques to assess how well the models perform. The most widely used measurement of the language model performance is called _perplexity_, and it is based on measuring how probable in the language the piece of text generated by a language model is. Let's implement this measure and evaluate the text generated by the language models above.

The first step is to collect the data which you will use to calculate probabilities. Use the reviews from either one subset or both subsets of the training data:


In [27]:
import os

def training_data(corpus, a_dir):
    file_list = os.listdir(a_dir)
    for a_file in file_list[:100]:
        f = open(a_dir + a_file, 'r')
        corpus += f.read()
        f.close()
    return corpus

training_corpus = training_data("", "data/imdb_dataset/pos/")
print(len(training_corpus))
#training_corpus = training_data(training_corpus, "imdb_dataset/neg/")
#print(len(training_corpus))

131105


Remember that the data read in this way contains string of symbols (characters) and before calculating the probabilities of words, you need to tokenise it into constituent words. Let's use spaCy as before:


In [28]:
import spacy
#################nlp = spacy.load('en') #################### it doesnt work!!
nlp = spacy.load('en_core_web_sm')

tokens = nlp(training_corpus)
print(len(tokens))

27018


The language model that you built earlier is very simple – it predicts the next character based on _character n-grams_, i.e. the previous _n_ characters in the sequence. It would be reasonable to evaluate whether it generates a sequence of legitimate English words. For that, you can use _word unigrams_ and estimate whether the language model based on character prediction generates highly probable English words.

**Note:**
>In practice, _word_ _bi-_ and _tri-grams_ are more frequently used to evaluate text prediction models.

Let's collect the statistics on the probability of words from the small training corpus you read in above. In essence, the function below iterates through the word tokens, calculates the number of occurrences for each token and divides it by the total number of occurrences for all word tokens in the corpus assigning probability to the words, i.e. if there are $n$ different word tokens (unigrams) in the corpus, probability of a particular word token (unigram) $P(w)$ is defined as:

\begin{equation}
P(w) = \frac{num(w)}{\sum_{i=1}^{n}num(w_{i})}
\end{equation}

In addition, however big the input corpus is, it is hard to expect that you will come across all possible words in this corpus. In practice, it is a good idea to reserve some small amount of probability for the words unseen in the training data, e.g. $\lambda=0.001$. Remember, you have come across this in the previous module: the technique of reserving some small probability for yet unseen events is called _smoothing_.

In [29]:
def unigram(tokens):    
    model = defaultdict(lambda: 0.001)
    for token in tokens:
        token = str(token).strip()
        try:
            model[token] += 1
        except KeyError:
            model[token] = 1
            continue
    total = 0
    for word in model:
        total+= model.get(word)
    for word in model:
        model[word] = model.get(word)/float(total)
    return model

Let's now calculate _perplexity_. For a string of $N$ words, perplexity is estimated as:

\begin{equation}
PP(W) = PP(w_{1}w_{2}...w_{N}) = \sqrt[N]{\frac{1}{P(w_{1}w_{2}...w_{N})}} = \sqrt[N]{\prod_{i=1}^{N}\frac{1}{P(w_{i})}}
\end{equation}



In [30]:
def perplexity(testset, model):
    testset = nlp(testset)
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[str(word).strip()])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

Note, that a good language model will generate a highly probable sequence of words. Since the probability is used in denominator, the better the language model, the higher the probability of the sequence – the lower the perplexity value. When you use perplexity to measure the quality of and to compare several language models, you are looking for the one that has **lower perplexity**.

With all the components in place, let's apply this measurement and compare a language model that generates text using previous 6 characters with the one that uses previous 8 characters. The more likely the words generated by the language model are, the lower the perplexity score.


In [31]:
lm_data = read_data("data/imdb_dataset/pos/")
model = unigram(tokens)

lm1 = LanguageModel(order=6)
lm1.train(lm_data)
testset1 = lm1.generate('the be', 280)
print(testset1)
print(perplexity(testset1, model))

lm2 = LanguageModel(order=8)
lm2.train(lm_data)
testset2 = lm2.generate('the best', 280)
print(testset2)
print(perplexity(testset2, model))

the beginning , multiple flawless amount would events that he really surnament. If Busy does there, but like a little messanger left me purring Jean Eustache drew member of Peggy grimaces and figures of Chris textures the origines make some occasional quality unraveling often deeper. M
800.8826384248341
the best gifts he's ever been very busy for Abe - in "Virginia City" he grants a pardon to Errol Flynn at the life-style of acting, then see why it's snobbish to convincing Sally Kellerman (who's idea was the baddies and stories Â– a whole series progressing on the story lines and effect
439.3490207794224


**Bonus exercise:**
>Implement a _word bi-gram_ based evaluation.

# The meaning of the word and where to find it

The language model above worked reasonably well: note that it is a very simple implementation that relies on just about $100$ lines of Python code, yet it can generate text that, in the first approximation, looks like proper English. Indeed, it follows the rules of English and is mostly grammatically correct. However, presented with such a text one will quickly recognise that the text is written by a machine rather than a human being – that is, the language model that you used will not pass the [Turing test](https://en.wikipedia.org/wiki/Turing_test). 

**Quiz:**
>What is the problem with this approach?

The main problem with the language model as implemented above is that it doesn't preserve the _meaning_, especially when it comes to longer pieces of text. Depending on the particular run, your language model might have generated perfectly sensible short expressions like "_the best part_", "_very engaging_", "_It's well worth_" and the like, but combined all together in a sentence or a whole review they rarely contribute to building a coherent story or review. In other words, the model fails to build _global meaning_. 

**Quiz:**
>Can you explain why?

In fact, this result is not very surprising: if you look closely into the code, the simple language model above never took the meaning of the words into account in the first place. All it does is trying to "copy" what is most probable sequence of characters from the data that you use to train it on. In this module, you will look into how to improve this technology and account for the meaning of words, sentences and even whole texts.


## Semantics

Whenever we are talking about the meaning of words, phrases, sentences and other "building blocks", we are talking about [_semantics_](https://en.wikipedia.org/wiki/Semantics). Semantics is the study of meaning and it deals with how we can represent the meaning of the smaller units such as senquences of characters or words, and how we can derive the meaning of larger units such as phrases and sentences, either on their own or using the meaning of the smaller building blocks. 

For example, if we agree that the meaning of [_pig_](https://en.wiktionary.org/wiki/pig) is that related to "any of several intelligent mammalian species of the genus Sus, having cloven hooves, bristles and a nose adapted for digging", and the meaning of [_small_](https://en.wiktionary.org/wiki/small) is that of "not large or big; few in numbers or size", then we can derive the meaning of the phrase _small pig_ by directly combining the meanings of the two building blocks (words) and say that _meaning("small pig") = meaning("small") + meaning("pig")_. This simple technique works quite well on many occasions, and arguably represents how we, humans, derive the meaning of larger expressions: we combine the meanings of the building blocks (words or concepts) where those are compatible. This also suggests that if we know how to represent the meaning of the words computationally, we can always derive the meaning of larger expressions up to sentences and even whole texts computationally or automatically.

However, human language and cognition are tricky matters and the simple approach above is not without its weaknesses: for example, the meaning of [_guinea pig_](https://en.wiktionary.org/wiki/guinea_pig) is not a simple combination of the meanings of [_guinea_](https://en.wiktionary.org/wiki/guinea) and _pig_ – in fact, it is an altogether different type of animals unrelated to pigs. This is one of the examples where the simple approach described above will fail. To make things even worse, _guinea pig_ is a building block in itself, and depending on the context of use can mean different things: apart from the animal sense, it can also mean "a living experimental subject" well applicable to humans. Words as well as phrases quite often have a variety of meanings – the phenomenon known as [_polysemy_](https://en.wikipedia.org/wiki/Polysemy). Which of the multiple meanings of a word are activated in each particular situation depends on the context of use: we, humans, tend to build mental meaning representations on the go as the story unfolds, however it is much trickier for the machines. It is so challenging that NLP experts even devoted a whole line of research to the task of distinguishing between multiple meanings of words and detecting which one is activated in each particular context of use – the task is called [Word Sense Disambiguation](https://en.wikipedia.org/wiki/Word-sense_disambiguation). At the same time, [_ambiguity_](https://en.wikipedia.org/wiki/Ambiguity) is often used on purpose in creative writing, for rhetorical effect, in jokes and so on: e.g., _“Can you make me a sandwich?” – “Abracadabra! You’re a sandwich!”_.

Figurative and non-literal uses of language are especially challegning for the machines to deal with: to wrap up with the pig-related examples, [_pig_](https://en.wiktionary.org/wiki/pig) can be used informally to denote "a difficult problem". Finally, sarcasm changes the original meaning of the expressions completely: _"Well done!"_ or _"What a surprise!"_ might look positive on the paper, but depending on the tone and the situation might mean completely opposite things, and be addressed to a person who did something wrong or used to describe a situation that you expected, predicted or even warned someone about.

There are many real-world applications of semantic models. For instance, semantics is used to find relevant information when you submit your queries to the search engine, to recommend you books and movies similar to your preferences, to find duplicates in a set of documents, to automatically generate sensible conversations (think about Siri and Alexa), and so on. The sentiment analyser you implemented earlier didn't use any notion of meaning but it can also benefit from the use of semantic models: for example, if a review contains many occurrences of the word "_bad_" it would be classified as a negative review, but what if all those were used in the context of "_not bad_"?


## Words as vectors

To teach machines understand human language we need to be able to represent meaning computationally. How should we do that?

The field has seen a breakthrough in teaching machines understand human language in 2013 with the publication of two papers by [Tomas Mikolov and his colleagues](https://arxiv.org/pdf/1310.4546.pdf) where they show how to efficiently represent word and phrase meaning computationally. In particular, [one of these papers](https://www.aclweb.org/anthology/N13-1090) presents a now famous example on a word analogy task: if the computer knows that the word _man_ stands in some relation to the word _woman_, given another input word, for example _king_, it can apply the same relation to it and retrieve the word that stands in the same relation to _king_, thus answering the analogy question "_Man_ is to _woman_ as _king_ is to _ ?"

**Quiz:**
> What is the word that should fill in the gap in the question above? What is the relation between the concepts?

The most impressive contribution of this work is that the computer **does not know** and **is not being told** what type of relation holds between the two words – it learns it independently by itself using lots of training examples, and having acquired this knowledge can apply this (and many other types of relations) to different pairs of words such as _king_-_queen_, _uncle_-_aunt_, _brother_-_sister_, and so on. The algorithm boils down to:

- knowing semantic representations of a wide variety of words, for example `rep(man)`, `rep(woman)`, `rep(king)`, `rep(queen)`, and so on
- putting these word representations in a shared semantic space 
- applying simple algebraic operations to the word representations: e.g., it has been shown that the result of the algebraic operations in the semantic space applied to `rep(king) - rep(man) + rep(woman)` is `rep(queen)`.

It can also be argued that our brains implicitly perform similar sort of operations: when looking for the answer for the analogy question above we take the `meaning(king)`, detract the `meaning(man)`, and add `meaning(woman)`, because we know that the word we are looking for shares all the properties of the word _king_ apart from those that distinguish between _man_ and _woman_. Now the central question is what the semantic representations are.

**Quiz:**
> How do we know who/what the queen is?

**Answer:**
> We acquire this knowledge over time using rich sources of audio and visual information: we read about kings and queens, we hear aboth them on the TV, the luckiest of us might meet with them personally, and so on.

![Who is the queen](figures/queen.png)

To teach the computer about the meaning of _king_ and _queen_ we need to provide it with the similar type of information – textual and visual descpription of what it means to be a king or a queen. Since we focus on NLP in this course, we will look into how the computers can learn who a queen is from text.

Even though the [Mikolov et al.'s paper](https://www.aclweb.org/anthology/N13-1090) was in many respects groundbreaking, the idea of representing word meaning computationally was not new. One of the most widely cited quotes in semantics is attributed to [John Firth](https://en.wikiquote.org/wiki/John_Rupert_Firth) who postulated back in 1957 (i.e., almost 60 years before that) that "_You shall know a word by the company it keeps_". Ever since, it has been the conrnerstone of computational semantics – or the way we represent the meaning of words for computers. In essence, what it says is that to define the meaning of a word or a concept, all we need to do is collect many contexts of use and derive the properties of the word from the neighbouring words.

For example, if we provide the computer with a lot of training data – many contexts of use of the word _queen_ – the computer will be able to learn that "crown", "royal", "state", visit", "palace", "speech" and other are characteristic properties, attributes and actions of queens. At the same time, the machine will learn that the key difference between kings and queens lies in their gender – the queen will regularly be referred with "she" and "her", while the king will be referred to as "he" or "his".

![Contexts of use for queen](figures/queen_contexts.png)

A very convenient and efficient way to represent word meaning then is _multi-dimensional vector space_ where each dimension represents a different property of the concept (for example, _royalty_ or _femininity_). For decades, computational semantics has been representing semantic space as a multi-dimensional vector space and benefitted from the following properties of such a representation:

- **generalisability**: as we pointed out above, it is important to be able to put all words and concepts in the universe in the same space, as it helps to interpret them and establish the relations
- **interpretability**: each dimension in the shared semantic space can be interpreted as a particular property so we can easily describe the concepts as collections of certain properties
- **visualisation**: vector representations also allow us to interpret the semantic space geometrically. This ranges from visualising the concepts to actually estimating similarity between words using geometrical measures.

For example, let's see how the different concepts will be represented in the vector-based semantic space. _Kings_ and _queens_ will share the property of being royal unlike _ministers_:

![Royals vs non-royals](figures/royals-vs-nonroyals.png)

but at the same time will be differentiated by the gender-related properties:

![Males vs females](figures/males-vs-females.png)

As we can only visualise up to 3 dimensions in practice, the above are very limited representations of words in the semantic space – try to imagine these in hundreds of thousands of dimensions! These representations, however, visualiase another useful property of the semantic spaces – **similar concepts are located closer together along the relevant dimensions than dissimilar concepts**. For example, _queen_ and _princess_ are located close together because they are both female and share a good deal of semantic meaning, but they will be more distant in space along the age-related dimension. Thus, if two concepts are **very similar** in meaning they will be located close to each other in the space. This gives us the theoretical foundation for how to detect similarity in meaning between words, as well as a practical tool to do that – as we said before, we can interpret the semantic space geometrically and calculate the distances in the semantic space as we would do in the geometric space.

Mikolov et al. coined the term _word embeddings_ which are, essentially, word vectors of $300$ dimensions. The developed tool [`word2vec`](link_to_word2vec) is freely available. It arrives at the word embeddings using one of the two regimes: the algorithm builds the target word vector using the context of the surrounding words (`cbow` model predicts the word given its context) or it builds the surrounding word vectors using the input word vector (`skip-gram` model predicts the context given a word):

![Skip-gram vs cbow](figures/skipgram-cbow.png)

Either way, the resulting vectors are less interpretable: the $300$ dimensions have been empirically found to work well, but each dimension doesn't represent a particular aspect of meaning as with the more traditional and more transparent word vectors. However, the vectors are of high quality – the fact that surrounding contexts are used to learn word vectors guarantees that similar words end up having similar semantic representations, so word embeddings have been demonstrated to perform exceptionally well on many semantic tasks.


## How to measure similarity

The first task that was used to demonstrate usefulness of word embeddings is the word analogy task: _man is to woman what king is to queen_. Once we have the semantic representations for _man_, _woman_ and _king_, the task boils down to two types of operations in the semantic space:

- **simple algebraic operations** of detraction [$vector(king)-vector(man)$] and addition [$+ vector(woman)$] on word vectors
- **search for the most similar word vector** to the resulting one.

The first step is easy – you need to simply detract and sum up the components of the word vectors along each dimension: e.g., if $\vec{a} = (3, 4)$ and $\vec{b} = (1, 2)$ then $\vec{a} - \vec{b} = (2, 2)$ and $\vec{a} + \vec{b} = (4, 6)$. Let's see now how to find the most similar word vector in the semantic space.

Since semantic space allows to apply geometric interpretation, we can measure the similarity through the shortest distance in the semantic space and use the approaches that will apply to measuring distances in Eucledean space. In particular, to measure the distance between vectors $\vec a$ and $\vec b$ we need to calculate the _cosine_ using the formula:

\begin{equation}
cos(\vec{a}, \vec{b}) = \frac{\vec{a} \cdot \vec{b}}{\lvert \lvert \vec{a} \rvert \rvert_{2} \lvert \lvert \vec{b} \rvert \rvert_{2}} = \frac{\sum_{i} (a_{i} \times b_{i})}{\sqrt{\sum_{i}^{n}(a_{i}^{2})} \times \sqrt{\sum_{i}^{n}(b_{i}^{2})}}
\end{equation}

<img src="figures/cos.png" width="350">

**Quiz:**
>What is $cos(\vec{a}, \vec{b})$, if $\vec{a} = (3, 4)$ and $\vec{b} = (1, 2)$?

**Answer:**
>\begin{equation}
cos(\vec{a}, \vec{b}) =
\frac{\vec{a} \cdot \vec{b}}{\lvert \lvert \vec{a} \rvert \rvert_{2} \lvert \lvert \vec{b} \rvert \rvert_{2}} =
\frac{\sum_{i} (a_{i} \times b_{i})}{\sqrt{\sum_{i}^{n}(a_{i}^{2})} \times \sqrt{\sum_{i}^{n}(b_{i}^{2})}} = 
\frac{3 \times 1 + 4 \times 2}{\sqrt{3^{2} + 4^{2}} \times \sqrt{1^{2} + 2^{2}}} = 
\frac{3 + 8}{\sqrt{25} \times \sqrt{5}} = 
\frac{11}{\sqrt{125}} \approx 0.98
\end{equation}

Let's translate these equations into Python code and check whether the answer is correct. For that, you'll use functionality of `numpy` and `math`:

In [32]:
import numpy as np

#Implement cosine similarity
# add your code here to define a function cosine that takes two vectors and returns the similarity
cosine = lambda v1, v2: np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2))

#Initialise vectors
a = np.fromstring('1 2', dtype=int, sep=' ')
b = np.fromstring('3 4', dtype=int, sep=' ')

print(cosine(a, b))

0.9838699100999074


## Word vectors with `spaCy`

Let's now write some code to access word vectors and measure semantic similarity between words. `spaCy` has nice functionality around that.

Let's start by measuring similarity between some sample words. Feel free to experiment with your own ones.

**Note**:
> You can import the set of models for English that you used for previous modules:

>>`import spacy`

>>`nlp = spacy.load('en')`

> Alternatively, you can use [state-of-the-art models](https://spacy.io/usage/models) trained on Web data for better performance. You'll need to first download them as:

>>`python -m spacy download en_core_web_md`

> or:

>>`pip install spacy`

>>`spacy download en_core_web_md`

> Once this is done, you can use the models importing the one you'd like: 

>>`import en_core_web_lg`

>>`nlp = en_core_web_lg.load()`

> or:

>>`import spacy`

>>`nlp = spacy.load(''en_core_web_lg')`

> for the large-size Web-trained model, `en_core_web_md` for the medium-size one, and `en_core_web_sm` for the small one. The default that is uploaded with `nlp = spacy.load('en')` uses the `en_core_web_sm`.

>The difference between the models is largely in performance (expect better results with larger models) and the space requirements: small model takes 35MB while large model takes 812MB. Check [documentation](http://spacy.io/models/en) to make up you mind about the choice of the model.

In [33]:
# Import spaCy and initialise the set of models for English
import spacy
# nlp = spacy.load('en')
nlp = spacy.load('en_core_web_md')
###############################nlp = spacy.load('en_core_web_sm') #### CHANGED to MD!!

text = u'box cat dog apple orange pasta pizza coffee tea' ###################### INCLUDED "BOX"!!!!!!!!!!!!!
words = nlp(text)

print("\t" + text.replace(" ", "\t"))

# Print out word similarities in a table
for word1 in words:
    output = str(word1) + "\t"
    for word2 in words:
        output += str(round(word1.similarity(word2), 4)) + "\t"
    print(output)

	box	cat	dog	apple	orange	pasta	pizza	coffee	tea
box	1.0	0.3235	0.2777	0.3119	0.3377	0.2454	0.2852	0.3636	0.3115	
cat	0.3235	1.0	0.8017	0.2821	0.3288	0.201	0.2767	0.2722	0.2802	
dog	0.2777	0.8017	1.0	0.2634	0.2743	0.2291	0.3503	0.287	0.294	
apple	0.3119	0.2821	0.2634	1.0	0.5619	0.3593	0.3902	0.4266	0.4289	
orange	0.3377	0.3288	0.2743	0.5619	1.0	0.3344	0.3134	0.3907	0.4201	
pasta	0.2454	0.201	0.2291	0.3593	0.3344	1.0	0.737	0.4176	0.3527	
pizza	0.2852	0.2767	0.3503	0.3902	0.3134	0.737	1.0	0.4977	0.3431	
coffee	0.3636	0.2722	0.287	0.4266	0.3907	0.4176	0.4977	1.0	0.746	
tea	0.3115	0.2802	0.294	0.4289	0.4201	0.3527	0.3431	0.746	1.0	


Note that the table is symmetric around the diagonal: that is, `cat.similarity(dog)=dog.similarity(cat)`. The highest value the similarity score can get is $1.0$, and since each word is most similar to itself the diagonal is filled with $1$'s. The next highest similarity score for this set of words is between _cat_ and _dog_. Note that other similarity scores tell us that _pizza_ and _pasta_, and _coffee_ and _tea_ are quite similar to each other – this follows our intuitions, since the first pair describes food and the second describes drinks. 

You can check your intuitions about word similarity comparing different words. For example, are _cats_ more similar to _dogs_ than they are to _boxes_? (Feel free to check similarity for your own words.)

![Cat or dog?](figures/cat-dog.png)


In [34]:
statement = u'Cat and dog are similar. Cat and frog aren\'t.'
text = nlp(statement.lower())
cat = text[0]
dog = text[2]
frog = text[8]
print ("Is statement \"" + statement + "\" correct?")
print(cat.similarity(dog))
print(cat.similarity(frog))
print(cat.similarity(dog) > cat.similarity(frog)) ############## WITH MODEL "SM" THE RESULT WAS FALSE!!!

Is statement "Cat and dog are similar. Cat and frog aren't." correct?
0.80168545
0.43867078
True


If you would like to 'look under the hood' and see how the word vectors are represented in `spaCy`, you can print them out directly. The code below prints out the first $10$ out of $300$ dimensions: the numbers are not directly interpretable, but note that those for `cat` and `dog` are indeed more similar to each other than to those for `frog`.


In [79]:
print (cat.vector[:10])
print (dog.vector[:10])
print (frog.vector[:10])

[-0.15067  -0.024468 -0.23368  -0.23378  -0.18382   0.32711  -0.22084
 -0.28777   0.12759   1.1656  ]
[-0.40176   0.37057   0.021281 -0.34125   0.049538  0.2944   -0.17376
 -0.27982   0.067622  2.1693  ]
[ 0.18531   0.11176  -0.38419  -0.41473  -0.24219   0.37046  -0.42745
 -0.13813  -0.080212  0.58181 ]


## Word analogy task with `spaCy`

Now, let's try to code the word analogy task and see if our algorithm can come up with the solution similar to the one presented in Mikolov et al.'s [paper](https://arxiv.org/pdf/1310.4546.pdf):

<img src="figures/countries.png" width="600">

That is, our analogy task will encode the relation `country:capital` but the computer won't be explicitly told that this is the relation to be used. Instead we'll ask a question _"Russia is to Moscow as China is to what?_ (as usual, feel free to insert your own variants). 

Let's first provide the list of countries and capitals in alphabetical order. To mix things up a bit, let's add some contries (e.g., _Switzerland_ and _Brazil_) with no corresponding capitals on the list, some capitals (e.g., _Amsterdam_ and _London_) with no corresponding countries, and some cities (e.g., _Barcelona_ and _Venice_) that are not capitals. You can always check whether the model has a vector for the word by printing out part of the word vector (always a good idea to check the data you are working with!):


In [35]:
text = u'Amsterdam Ankara Athens Australia Barcelona Beijing Berlin Brazil Chicago China '
text += u'France Germany Greece Italy Japan Lisbon London Madrid Moscow Paris '
text += u'Poland Portugal Rome Russia Spain Switzerland Tokyo Turkey Venice Warsaw '
words = nlp(text)

for word in words:
    print(word)
    print (word.vector[:5])

Amsterdam
[ 0.41373  -0.095219  0.15888  -0.51876   0.41066 ]
Ankara
[ 0.10742  -0.39872   0.41782   0.80667  -0.022942]
Athens
[ 0.21507   0.080389  0.25134  -0.3766    0.49439 ]
Australia
[-0.19401  0.44029  0.18968 -0.52768  0.70956]
Barcelona
[ 0.36657 -0.29327  0.16112  0.23317  0.54403]
Beijing
[ 0.78468  0.10513  0.13023 -0.28823  0.19203]
Berlin
[0.243   0.11444 0.36875 0.2368  0.57666]
Brazil
[-0.067526  0.050342  0.59258   0.41409   0.6952  ]
Chicago
[ 0.041337 -0.027126  0.28624  -0.27172   1.0035  ]
China
[-0.55554   0.32522  -0.090127 -0.49263   0.57665 ]
France
[-0.16306  0.45292 -0.14638 -0.64332  0.79014]
Germany
[-0.35532    0.59025    0.17082   -0.0029313  0.73128  ]
Greece
[ 0.21507   0.080389  0.25134  -0.3766    0.49439 ]
Italy
[-0.21052    0.18476   -0.0056243 -0.15168    0.78708  ]
Japan
[-0.44528   -0.17553    0.075346   0.0048481  0.2341   ]
Lisbon
[ 0.30521 -0.15312 -0.12796  0.42254  0.76491]
London
[ 0.37837  -0.32807   0.55623  -0.050013  0.72532 ]
Madrid
[

Now you are all set to try out the analogy task.

In [36]:
question = u"Russia is to Moscow as China is to WHAT?"
text = nlp(question)
source1 = text[0]
source2 = text[3]
target1 = text[5]

max_sim = 0.0
target2 = "N/A"

#Apply the operations on vectors
target2_vector = source2.vector-source1.vector+target1.vector

#Find the word with the most similar vector to the result
for word in words:
    if not (str(word)==str(target1) or str(word)==str(source1) or str(word)==str(source2)):
        current_sim = cosine(target2_vector, word.vector)
        if current_sim >= max_sim:
            max_sim = current_sim 
            target2 = word

print(question)
print(target2)   ############## WITH MODEL "SM" THE RESULT WAS FALSE!!!

Russia is to Moscow as China is to WHAT?
Beijing


Excellent! Now, let's run the task on all countries:

In [37]:
#Define analogy task as a separate method
#Note that the code below is almost exactly the same
def analogy_task(country):
    question = u"Russia is to Moscow as " + country
    text = nlp(question)
    source1 = text[0]
    source2 = text[3]
    target1 = text[5]

    max_sim = 0.0
    target2 = "N/A"

    target2_vector = source2.vector-source1.vector+target1.vector

    for word in words:
        if not (str(word)==str(target1) or str(word)==str(source1) or str(word)==str(source2)):
            current_sim = cosine(target2_vector, word.vector)
            if current_sim >= max_sim:
                max_sim = current_sim 
                target2 = word

    print(question)
    print("\t is to " + str(target2))
    

countries = ["China", "France", "Germany", "Greece", "Italy", 
             "Japan", "Poland", "Portugal", "Spain", "Turkey"]

for country in countries:
    analogy_task(country)

Russia is to Moscow as China
	 is to Beijing
Russia is to Moscow as France
	 is to Paris
Russia is to Moscow as Germany
	 is to Berlin
Russia is to Moscow as Greece
	 is to Athens
Russia is to Moscow as Italy
	 is to Rome
Russia is to Moscow as Japan
	 is to Tokyo
Russia is to Moscow as Poland
	 is to Warsaw
Russia is to Moscow as Portugal
	 is to Lisbon
Russia is to Moscow as Spain
	 is to Barcelona
Russia is to Moscow as Turkey
	 is to Ankara


Does the result correspond to the real state-of-affairs? 

**Bonus:**
>Apply the analogy task to pairs of words linked with other types of relations. For inspiration, consider the following examples from Mikolov et al.'s [paper](https://arxiv.org/pdf/1301.3781.pdf):

<img src="figures/other_relations.png" width="600">


## Document similarity with `spaCy`

Now, imagine you have submitted a review to the IMDB and would like to see similar reviews (similarly positive or similarly negative) about a particular movie, or perhaps, you'd simply like to see reviews from like-minded viewers – e.g., similarly positive or negative reviews on similar movies.

You can find reviews similar in content and sentiment to yours by applying the `similarity` function to the whole review and finding the other review with the overall highest similarity score to the input one. The code below shows how you can do that. To speed processing up, let's consider only a hundred of the positive and a hundred of the negative reviews. Pick up an input review at random (in the example below, `imdb_dataset/pos/989_9.txt` is used). Note that it is hard to expect that the algorithm will identify a review with a similar sentiment on the *same* movie – after all, the dataset is very sparse with respect to the movies covered! However, let's check whether the algorithm is capable of finding the review with _similar content and sentiment_. In an actual application, this algorithm can be applied to the set of reviews written on the same movie, which will result in finding similar reviews on the same movie across the users.


In [8]:
import os
import string
import spacy
################################nlp = spacy.load('en')
nlp = spacy.load('en_core_web_md') ##################################### CHANGED!!


def init_dict(a_dir):
    a_dict = {}
    file_list = os.listdir(a_dir)
    for a_file in file_list[:100]:
        if not a_file.startswith("."):
            f = open(a_dir + a_file, 'r')
            a_dict[a_file] = nlp(f.read())
            f.close()
    return a_dict


pos = init_dict("data/imdb_dataset/pos/")
neg = init_dict("data/imdb_dataset/neg/")

f = open("data/imdb_dataset/pos/989_9.txt", 'r')
review = nlp(f.read())
f.close()

max_sim = 0.0
similar = "N/A"
similar_path = "N/A"
for rev in pos.keys():
    rev_text = pos.get(rev)
    current_sim = review.similarity(rev_text)

    print(current_sim)
    
    if current_sim >= max_sim:
        max_sim = current_sim
        similar = rev_text
        similar_path = "pos/" + rev

print('========')
print('NEGATIVE')
print('========')

for rev in neg.keys():
    rev_text = neg.get(rev)
    current_sim = review.similarity(rev_text)

    print(current_sim)    
    
    if current_sim >= max_sim:
        max_sim = current_sim
        similar = rev_text
        similar_path = "neg/" + rev

        
print("The most similar review to:\n")
print(review[:1000])
print("\nis:\n")
print(similar[:1000])
print(similar_path)

0.9695912697908643
0.9567454875270055
0.9686506430668579
0.9797474416415846
0.9719893723563529
0.9893333062405464
0.9856507495199566
0.9681738785516374
0.9807053018827281
0.9557704460124985
0.9712462168047581
0.9690012352389686
0.9534977457772341
0.9550070286057575
0.9559348805549158
0.9771258841199503
0.9752825092608302
0.9381117380344393
0.969137981930322
0.9652884340258469
0.9708503020533172
0.9626264322035399
0.959151782298883
0.9765290112860704
0.9799965214822864
0.9812361404315019
0.9754487662397097
0.9795163788597694
0.9693143454070284
0.9770310532932374
0.9701460688817009
0.9685751198800868
0.9793509813078454
0.9602109265070402
0.9525605770440329
0.9806463081701483
0.9641500958687438
0.966294542387178
0.9698115425502075
0.9803168731197518
0.9493309098773883
0.984583202864715
0.9876059386484985
0.979999486958209
0.9770760019827816
0.9681457765500985
0.9364873956393414
0.955160883350104
0.9861277979992421
0.9839599348291671
0.9648777880203256
0.9626228704825024
0.9762049909630514

In [5]:
rev_text

This is an art film that was either made in 1969 or 1972 (the National Film Preservation Foundation says 1969 and IMDb says 1972). Regardless of the exact date, the film definitely appears to be very indicative of this general time period--with some camera-work and pop art stylings that are pure late 60s-early 70s.  The film consists of three simple images that are distorted using different weird camera tricks. These distorted images are accompanied by music and there is absolutely no dialog or plot of any sort. This was obviously intended as almost like a form of performance art, and like most performance art, it's interesting at first but quickly becomes tiresome. The film, to put it even more bluntly, is a total bore and would appeal to no one but perhaps those who made the film, their family and friends and perhaps a few people just too hip and "with it" to be understood by us mortals.

In [6]:
review.similarity(rev_text)

0.9721918196557798

In [1]:
#### https://dzone.com/articles/how-related-your-documents-are-measure-document-si
import glob
import os
file_list = glob.glob(os.path.join(os.getcwd(), "doccomparison/", "*.txt"))
raw_documents = []
for file_path in file_list:
    with open(file_path, encoding="utf8") as f_input:
        raw_documents.append(f_input.read())
print("Number of documents:",len(raw_documents))

Number of documents: 5


In [2]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
print(gen_docs)

[['data', 'science', 'sits', 'at', 'the', 'core', 'of', 'any', 'analytical', 'exercise', 'conducted', 'on', 'a', 'big', 'data', 'or', 'internet', 'of', 'things', '(', 'iot', ')', 'environment', '.', 'data', 'science', 'involves', 'a', 'wide', 'array', 'of', 'technologies', ',', 'business', ',', 'and', 'machine', 'learning', 'algorithms', '.', 'the', 'purpose', 'of', 'data', 'science', 'is', 'just', 'not', 'doing', 'machine', 'learning', 'or', 'statistical', 'analysis', 'but', 'also', 'to', 'derive', 'insights', 'out', 'of', 'the', 'data', 'that', 'a', 'user', 'with', 'no', 'statistics', 'knowledge', 'can', 'understand', '.', 'in', 'a', 'fast', 'paced', 'environment', 'such', 'as', 'big', 'data', 'and', 'iot', 'where', 'the', 'type', 'of', 'data', 'might', 'vary', 'over', 'the', 'course', 'of', 'time', ',', 'it', 'becomes', 'difficult', 'to', 'maintain', 'and', 'recreate', 'the', 'models', 'each', 'and', 'every', 'time.this', 'gap', 'calls', 'up', 'for', 'an', 'automated', 'way', 'to', 

In [3]:
import gensim
dictionary = gensim.corpora.Dictionary(gen_docs)

In [4]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

[[(0, 1), (1, 2), (2, 2), (3, 27), (4, 49), (5, 7), (6, 1), (7, 1), (8, 43), (9, 1), (10, 1), (11, 1), (12, 1), (13, 5), (14, 6), (15, 5), (16, 4), (17, 7), (18, 1), (19, 1), (20, 1), (21, 29), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 11), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 8), (34, 1), (35, 3), (36, 1), (37, 1), (38, 4), (39, 16), (40, 3), (41, 3), (42, 10), (43, 1), (44, 1), (45, 3), (46, 6), (47, 1), (48, 1), (49, 1), (50, 3), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 6), (60, 1), (61, 9), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 3), (94, 1), (95, 1), (96, 1), (97, 2), (98, 53), (99, 1), (100, 1), (101, 3), (102, 1), (103, 1), (104, 1), (105, 3), (106, 2), (107, 1), (108, 1), (109, 1), 

In [5]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

TfidfModel(num_docs=5, num_nnz=1970)


In [6]:
import gensim.downloader as api
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [7]:
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)


C:\Anaconda3\envs\ads9ldn\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity_matrix` (Method will be removed in 4.0.0, use gensim.models.keyedvectors.WordEmbeddingSimilarityIndex instead).
  """Entry point for launching an IPython kernel.


In [8]:
from gensim.matutils import softcossim
softcossim(corpus[0], corpus[1],similarity_matrix)

C:\Anaconda3\envs\ads9ldn\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  


0.9548263841157888

In [9]:
from gensim.matutils import softcossim
softcossim(corpus[0], corpus[3],similarity_matrix)

C:\Anaconda3\envs\ads9ldn\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  


0.9038845756420927

In [12]:
from gensim.matutils import softcossim
softcossim(corpus[0], corpus[0],similarity_matrix)

C:\Anaconda3\envs\ads9ldn\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  


1.0